# Model Creation: 

In [1]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import re

# Statistics
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats

# Sklearn - model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Sklearn - linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC

from sklearn.feature_selection import RFE

In [2]:
data = pd.read_csv('Life_Expectancy_clean.csv')
data.head()

,Life_Expectancy,State_FIPS_Code,County_FIPS_Code,Premature_death_raw_value,Poor_or_fair_health_raw_value,Poor_physical_health_days_raw_value,Poor_mental_health_days_raw_value,Low_birthweight_raw_value,Adult_smoking_raw_value,Adult_obesity_raw_value,...,Uninsured_children_raw_value,Other_primary_care_providers_raw_value,Median_household_income_raw_value,Children_eligible_for_free_or_reduced_price_lunch_raw_value,Residential_segregation___non_white/white_raw_value,Homeownership_raw_value,Severe_housing_cost_burden_raw_value,Population_raw_value,County_Ranked_(Yes1/No0),Drinking_water_violations_raw_value
0,72.438750,-1.694237,-0.703354,2.334237,0.585167,0.972298,0.744619,1.185344,1.262168,0.600135,...,-0.542966,-0.303066,-0.110904,0.526556,-0.593029,0.093544,0.719976,-0.353999,0.149924,-0.857549
1,70.426037,-1.694237,-0.686844,2.505048,2.389854,2.887774,1.915747,1.184023,2.510570,0.882490,...,-0.802039,0.220084,-1.719216,1.152966,1.602039,0.173994,-0.089176,-0.334493,0.149924,1.166115
2,75.056297,-1.694237,-0.670334,0.548239,0.430316,0.944534,0.598795,0.020882,1.220460,-0.082221,...,-0.248064,-0.477779,-0.568212,0.575578,-0.215187,0.787929,-0.730263,-0.304624,0.149924,-0.857549
3,77.644415,-1.694237,-0.653824,-0.211277,-0.007011,0.261949,0.311151,-0.090746,0.551838,0.811901,...,-0.932296,-0.799614,0.847341,-0.419564,-0.252510,1.220457,-0.453010,-0.053936,0.149924,-0.857549
4,74.386212,-1.694237,-0.637314,1.031308,1.168960,1.455216,1.086015,0.018399,1.406349,-0.199869,...,-0.360181,-0.272734,-0.276522,0.254347,0.807143,0.142721,-0.782571,-0.334972,0.149924,-0.857549


## Check Normality of Variables

In [3]:
# X.hist(figsize=(16,30))
# plt.show()

In [4]:
# County_Ranked_(Yes1/No0) is causing an issue therefore rename:
data.rename(columns={'County_Ranked_(Yes1/No0)': 'County_Ranked'}, inplace=True)

## Train Test Split

In [5]:
X_init = data.drop(columns=['Life_Expectancy'])
y_init = data['Life_Expectancy']

In [6]:
X_init.shape, y_init.shape

((2138, 56), (2138,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_init, y_init, test_size=1000, random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1138, 56), (1000, 56), (1138,), (1000,))

In [9]:
X = X_train
y = y_train

In [10]:
df_model = pd.concat([y,X], axis=1)
df_model.head()

,Life_Expectancy,State_FIPS_Code,County_FIPS_Code,Premature_death_raw_value,Poor_or_fair_health_raw_value,Poor_physical_health_days_raw_value,Poor_mental_health_days_raw_value,Low_birthweight_raw_value,Adult_smoking_raw_value,Adult_obesity_raw_value,...,Uninsured_children_raw_value,Other_primary_care_providers_raw_value,Median_household_income_raw_value,Children_eligible_for_free_or_reduced_price_lunch_raw_value,Residential_segregation___non_white/white_raw_value,Homeownership_raw_value,Severe_housing_cost_burden_raw_value,Population_raw_value,County_Ranked,Drinking_water_violations_raw_value
813,77.858539,-0.340367,-0.554765,-0.091870,0.816641,1.022773,0.444620,-0.461421,-0.130554,-1.705759,...,0.315627,-0.165096,-0.960806,0.853923,-1.183209,-1.286539,2.125187,-0.309893,0.149924,1.166115
1604,80.577406,0.916798,0.518378,-0.725498,-0.330078,-0.653514,-0.665404,-0.160177,-1.298671,-1.211639,...,3.778153,-0.371081,0.545436,-0.279159,0.298090,0.565569,0.576227,-0.277381,0.149924,1.166115
406,74.986425,-1.017302,-0.736374,0.751385,1.156605,0.566277,0.490213,1.691675,0.459604,0.929549,...,-0.359932,-0.500568,-0.975835,0.864885,0.478539,0.591215,-0.353150,-0.353953,0.149924,-0.857549
778,81.993454,-0.437072,-0.752884,-1.256101,0.688591,-0.015012,-0.560650,0.282981,-0.633726,-2.011643,...,-0.232505,-0.897902,1.075192,0.876656,0.328459,-4.940648,3.203709,2.494831,0.149924,-0.857549
1211,76.611541,0.239863,-0.769393,0.122952,-0.514874,0.116825,0.637500,-0.875349,-0.829082,-0.129280,...,-0.282384,0.147542,-0.755572,0.338099,-0.648674,-0.508717,0.850042,-0.293960,0.149924,1.166115


# Model 1:

- All variable carried forward from the cleaning

## Statsmodule OLS model

In [11]:
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const).fit()
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_Expectancy   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.918
Method:                 Least Squares   F-statistic:                     227.8
Date:                Tue, 14 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:09:08   Log-Likelihood:                -1428.5
No. Observations:                1138   AIC:                             2971.
Df Residuals:                    1081   BIC:                             3258.
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
const                                                          77.3898      0.026   2975.745      0.000      77.339      77.441
State_FIPS_Code                                                 0.0360      0.034      1.052      0.293      -0.031       0.103
County_FIPS_Code                                                0.0478      0.034      1.402      0.161      -0.019       0.115
Premature_death_raw_value                                      -0.4065      0.114     -3.567      0.000      -0.630      -0.183
Poor_or_fair_health_raw_value                                   0.2167      0.122      1.773      0.077      -0.023       0.457
Poor_physical_health_days_raw_value                            -0.5608      0.300     -1.867      0.062      -1.150       0.029
Poor_mental_health_days_raw_value                               0.0711      0.188      0.379      0.705      -0.297       0.439
Low_birthweight_raw_value                                      -0.0081      0.046     -0.177      0.860      -0.098       0.082
Adult_smoking_raw_value                                         0.0834      0.076      1.102      0.271      -0.065       0.232
Adult_obesity_raw_value                                        -0.0219      0.044     -0.500      0.617      -0.108       0.064
Food_environment_index_raw_value                               -0.6983      0.517     -1.351      0.177      -1.713       0.316
Physical_inactivity_raw_value                                  -0.0091      0.052     -0.175      0.861      -0.111       0.093
Access_to_exercise_opportunities_raw_value                     -0.0205      0.039     -0.532      0.595      -0.096       0.055
Excessive_drinking_raw_value                                   -0.0542      0.050     -1.082      0.279      -0.152       0.044
Alcohol_impaired_driving_deaths_raw_value                       0.0140      0.028      0.492      0.623      -0.042       0.070
Sexually_transmitted_infections_raw_value                       0.0460      0.039      1.192      0.234      -0.030       0.122
Teen_births_raw_value                                           0.0556      0.057      0.967      0.334      -0.057       0.168
Uninsured_raw_value                                             2.0297      0.659      3.082      0.002       0.737       3.322
Primary_care_physicians_raw_value                              -0.0666      0.041     -1.615      0.107      -0.148       0.014
Dentists_raw_value                                              0.0189      0.037      0.516      0.606      -0.053       0.091
Mental_health_providers_raw_value                              -0.0054      0.032     -0.167      0.867      -0.068       0.057
Preventable

<b>Observations</b>

- High R^2 value
- High AIC & BIC Values as lots of features

## K-fold / Cross Validation Model

In [12]:
regression = LinearRegression()

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring='r2', cv=crossvalidation))
print("Inital R^2:", baseline)

Inital R^2: 0.9011977538912742


<b>Observations</b>

- Very High R^2 value -> similar to the stats module value
- Look to reduce variables

# Drop Columns on Evaluation of; 
- Ease of collection 
- Quantifiability  
- Cost of Collection

In [13]:
drop = ['Premature_death_raw_value', 'Poor_or_fair_health_raw_value', 
        'Poor_physical_health_days_raw_value', 'Poor_mental_health_days_raw_value', 
        'Adult_smoking_raw_value', 'Physical_inactivity_raw_value', 'Excessive_drinking_raw_value',
       'Preventable_hospital_stays_raw_value', 'Mammography_screening_raw_value', 
        'Driving_alone_to_work_raw_value',
       'Long_commute___driving_alone_raw_value', 'Frequent_physical_distress_raw_value', 
        'Frequent_mental_distress_raw_value',
       'Insufficient_sleep_raw_value','Other_primary_care_providers_raw_value',
        'Drinking_water_violations_raw_value']
len(drop)

16

In [14]:
cols = list(X.columns)
new_cols = [x for x in cols if x not in drop]

In [15]:
X_1 = X[new_cols]
len(X.columns) , len(X_1.columns)

(56, 40)

In [16]:
regression = LinearRegression()

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X_1, y, scoring='r2', cv=crossvalidation))
print("Inital R^2:", baseline)

Inital R^2: 0.9015907092924367


In [17]:
X_const = sm.add_constant(X_1)

model = sm.OLS(y, X_const).fit()
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_Expectancy   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     305.9
Date:                Tue, 14 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:09:10   Log-Likelihood:                -1458.2
No. Observations:                1138   AIC:                             2998.
Df Residuals:                    1097   BIC:                             3205.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
const                                                          77.3894      0.026   2927.274      0.000      77.337      77.441
State_FIPS_Code                                                 0.0006      0.031      0.021      0.983      -0.060       0.061
County_FIPS_Code                                                0.0550      0.033      1.676      0.094      -0.009       0.119
Low_birthweight_raw_value                                      -0.0215      0.042     -0.511      0.610      -0.104       0.061
Adult_obesity_raw_value                                        -0.0354      0.041     -0.869      0.385      -0.115       0.044
Food_environment_index_raw_value                               -0.5708      0.523     -1.091      0.276      -1.598       0.456
Access_to_exercise_opportunities_raw_value                     -0.0535      0.038     -1.425      0.154      -0.127       0.020
Alcohol_impaired_driving_deaths_raw_value                       0.0125      0.028      0.439      0.661      -0.043       0.068
Sexually_transmitted_infections_raw_value                       0.0266      0.038      0.695      0.487      -0.048       0.102
Teen_births_raw_value                                           0.1034      0.056      1.859      0.063      -0.006       0.213
Uninsured_raw_value                                             1.8315      0.629      2.910      0.004       0.596       3.067
Primary_care_physicians_raw_value                              -0.0342      0.036     -0.962      0.336      -0.104       0.036
Dentists_raw_value                                              0.0232      0.037      0.635      0.525      -0.049       0.095
Mental_health_providers_raw_value                              -0.0118      0.031     -0.377      0.706      -0.073       0.050
Flu_vaccinations_raw_value                                     -0.0310      0.035     -0.891      0.373      -0.099       0.037
High_school_graduation_raw_value                                0.0745      0.033      2.250      0.025       0.010       0.139
Some_college_raw_value                                          0.0295      0.046      0.644      0.520      -0.060       0.119
Unemployment_raw_value                                          0.0949      0.044      2.162      0.031       0.009       0.181
Children_in_poverty_raw_value                                   0.0925      0.071      1.302      0.193      -0.047       0.232
Income_inequality_raw_value                                     0.2272      0.036      6.235      0.000       0.156       0.299
Children_in_single_parent_households_raw_value                 -0.0341      0.047     -0.721      0.471      -0.127       0.059
Social_asso

# Feature ranking to recursive eliminate features

In [18]:
# df_feat_rank = pd.concat([y,X_1], axis=1)
# df_feat_rank.head()

In [19]:
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=20) # 10 most important features
selector = selector.fit(X_1, y)
estimators = selector.estimator_

In [22]:
print(selector.support_ )
print(selector.ranking_)
print(estimators.coef_)
print(estimators.intercept_)

[False False False False  True  True False False  True  True False False
 False False  True False  True  True  True False False False  True  True
 False  True False False  True  True False  True  True  True  True False
  True  True False  True]
[21  5 17 11  1  1 19 16  1  1 12 18 20 14  1 13  1  1  1  8 15  2  1  1
  9  1 10  6  1  1  7  1  1  1  1  4  1  1  3  1]
[-0.4310896  -0.06468542  0.06428218  1.71434468  0.08622612  0.10872355
  0.10404052  0.2234006  -0.12617376 -0.1944719  -2.80965254 -0.45440699
 -0.07483478 -1.54095193 -0.22848109  0.21367407  0.11750662 -0.13509753
 -0.10370785  0.08385941]
77.3868631331854


In [23]:
X_cols = list(X_1.columns)
X_best = []
i=0
for x in selector.support_:
    if x == True:
        X_best.append((X_cols[i], selector.ranking_[i]))
    i+=1

In [24]:
# X_best

In [25]:
sell = list(zip(X_cols, estimators.coef_, selector.ranking_, selector.support_))
df_selection = pd.DataFrame(sell, columns=['X_cols', 'coef','ranking', 'support'])
df_selection.sort_values(by=['ranking'], inplace=True)
df_selection.head(30)

,X_cols,coef,ranking,support
9,Uninsured_raw_value,-0.194472,1,True
17,Children_in_poverty_raw_value,-0.135098,1,True
16,Unemployment_raw_value,0.117507,1,True
4,Food_environment_index_raw_value,0.086226,1,True
5,Access_to_exercise_opportunities_raw_value,0.108724,1,True
14,High_school_graduation_raw_value,-0.228481,1,True
18,Income_inequality_raw_value,-0.103708,1,True
8,Teen_births_raw_value,-0.126174,1,True
1,County_FIPS_Code,-0.064685,5,False
19,Children_in_single_parent_households_raw_value,0.083859,8,False


# Interactions 

In [16]:
def feature_combinations_r_sqrd_with_Inter_df(X, y, num_feat_comb=2):
    # Requires cals: baseline & crossvalidation
    
    # Create Regression & Combinations
    from itertools import combinations
    combinations = list(combinations(list(X.columns), num_feat_comb))
    
    # Create cross-validation & output a bassline MSE score as a DataFrame
    comb_scores = []
    inter_cols = []
    inter_score = []
    data = X.copy()
    
    for comb in combinations:
        data['interaction'] = data[comb[0]] * data[comb[1]]
        score = np.mean(cross_val_score(regression, data, y, scoring='r2', cv=crossvalidation))
        if score > baseline: 
            comb_scores.append(round(score,3))
            inter_cols.append((str(comb[0]) + '_' + str(comb[1])))
            inter_score.append(data[comb[0]] * data[comb[1]])
    
    df_base = pd.DataFrame(data=[inter_cols, comb_scores])
    df_base = df_base.T  
    df_base.rename(columns={0: "Interaction", 1: "CV_score"}, inplace=True)
    df_base.sort_values(by='CV_score', inplace = True, ascending=False )
    df_base.reset_index(drop=True, inplace = True)
    
    df_interactions_scores = pd.DataFrame(data=inter_score , index=inter_cols)
    df_interactions_scores = df_interactions_scores.T
    

    return df_base , df_interactions_scores

In [17]:
df_base, df_score = feature_combinations_r_sqrd_with_Inter_df(X,y)

In [18]:
df_base.head(10)

,Interaction,CV_score
0,Premature_death_raw_value_Premature_age_adjust...,0.911
1,Injury_deaths_raw_value_Premature_age_adjusted...,0.91
2,Adult_smoking_raw_value_Premature_age_adjusted...,0.91
3,Children_in_poverty_raw_value_Premature_age_ad...,0.909
4,Premature_death_raw_value_Adult_smoking_raw_value,0.909
5,Premature_death_raw_value_Injury_deaths_raw_value,0.908
6,Premature_death_raw_value_Frequent_physical_di...,0.908
7,Premature_age_adjusted_mortality_raw_value_Fre...,0.908
8,Premature_age_adjusted_mortality_raw_value_Fre...,0.908
9,Mammography_screening_raw_value_Income_inequal...,0.908


In [19]:
def add_interaction_feature(data, df_inter, df_score, num_inter):
    i=0
    
    while i < num_inter:
        col = df_inter['Interaction'][i]
  
        data[col] = df_score[col]
        i+=1
    
    return data

In [20]:
data_combined = add_interaction_feature(X, df_base, df_score, 10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
data_combined.head()

,State_FIPS_Code,County_FIPS_Code,Premature_death_raw_value,Poor_or_fair_health_raw_value,Poor_physical_health_days_raw_value,Poor_mental_health_days_raw_value,Low_birthweight_raw_value,Adult_smoking_raw_value,Adult_obesity_raw_value,Food_environment_index_raw_value,...,Premature_death_raw_value_Premature_age_adjusted_mortality_raw_value,Injury_deaths_raw_value_Premature_age_adjusted_mortality_raw_value,Adult_smoking_raw_value_Premature_age_adjusted_mortality_raw_value,Children_in_poverty_raw_value_Premature_age_adjusted_mortality_raw_value,Premature_death_raw_value_Adult_smoking_raw_value,Premature_death_raw_value_Injury_deaths_raw_value,Premature_death_raw_value_Frequent_physical_distress_raw_value,Premature_age_adjusted_mortality_raw_value_Frequent_mental_distress_raw_value,Premature_age_adjusted_mortality_raw_value_Frequent_physical_distress_raw_value,Mammography_screening_raw_value_Income_inequality_raw_value
813,-0.340367,-0.554765,-0.091870,0.816641,1.022773,0.444620,-0.461421,-0.130554,-1.705759,-1.195167,...,0.008252,0.028032,0.011727,-0.106116,0.011994,0.028669,-0.111111,-0.079626,-0.108639,-2.307872
1604,0.916798,0.518378,-0.725498,-0.330078,-0.653514,-0.665404,-0.160177,-1.298671,-1.211639,0.363873,...,0.631074,0.430578,1.129648,0.383412,0.942183,0.359123,0.470410,0.669600,0.564007,0.086777
406,-1.017302,-0.736374,0.751385,1.156605,0.566277,0.490213,1.691675,0.459604,0.929549,-0.329034,...,0.608975,0.348957,0.372495,0.975223,0.345340,0.323517,0.519896,0.560722,0.560777,-1.784178
778,-0.437072,-0.752884,-1.256101,0.688591,-0.015012,-0.560650,0.282981,-0.633726,-2.011643,1.143393,...,1.507524,2.258487,0.760573,-0.036626,0.796023,2.363756,-0.364737,0.656272,-0.348493,-1.696385
1211,0.239863,-0.769393,0.122952,-0.514874,0.116825,0.637500,-0.875349,-0.829082,-0.129280,0.104033,...,0.004065,0.091658,-0.027410,0.010430,-0.101937,0.340872,-0.016453,0.007520,-0.004424,-0.084900


# Polynomial Parameters

In [22]:
# from sklearn.preprocessing import PolynomialFeatures

# regression = LinearRegression()
# crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)

# polynomials = []
# for col in X.columns:
#     for degree in [2, 3, 4]:
#         data = X.copy()
#         poly = PolynomialFeatures(degree, include_bias=False)
#         X_transformed = poly.fit_transform(X[[col]])
#         data = pd.concat([data.drop(col, axis=1),pd.DataFrame(X_transformed)], axis=1)
#         score = np.mean(cross_val_score(regression, data, y, scoring='r2', cv=crossvalidation))
#         if score > baseline: polynomials.append((col, degree, round(score, 3)))
# print('Top 10 polynomials: %s' %sorted(polynomials, key=lambda poly: poly[2], reverse=True)[:10])

In [23]:
# polynom = pd.DataFrame(polynomials)
# polynom.groupby([0], sort=False)[2].max()

# Full model R-squared  

In [24]:
full_model = np.mean(cross_val_score(regression, data_combined, y, scoring='r2', cv=crossvalidation))
print("Full model R^2:", full_model)

Full model R^2: 0.9142523127551015


In [26]:
X_const = sm.add_constant(data_combined)

model = sm.OLS(y, X_const).fit()
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_Expectancy   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     236.4
Date:                Tue, 14 Jan 2020   Prob (F-statistic):               0.00
Time:                        10:11:07   Log-Likelihood:                -1317.2
No. Observations:                1138   AIC:                             2768.
Df Residuals:                    1071   BIC:                             3106.
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================================
                                                                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                              77.2219      0.033   2371.000      0.000      77.158      77.286
State_FIPS_Code                                                                     0.0161      0.032      0.508      0.612      -0.046       0.078
County_FIPS_Code                                                                    0.0468      0.031      1.497      0.135      -0.015       0.108
Premature_death_raw_value                                                          -0.3682      0.111     -3.311      0.001      -0.586      -0.150
Poor_or_fair_health_raw_value                                                       0.2433      0.113      2.161      0.031       0.022       0.464
Poor_physical_health_days_raw_value                                                -0.1855      0.280     -0.661      0.509      -0.736       0.365
Poor_mental_health_days_raw_value                                                   0.2759      0.177      1.560      0.119      -0.071       0.623
Low_birthweight_raw_value                                                           0.0140      0.042      0.333      0.739      -0.068       0.096
Adult_smoking_raw_value                                                             0.0605      0.072      0.846      0.398      -0.080       0.201
Adult_obesity_raw_value                                                            -0.0195      0.041     -0.481      0.631      -0.099       0.060
Food_environment_index_raw_value                                                   -0.8703      0.472     -1.843      0.066      -1.797       0.056
Physical_inactivity_raw_value                                                       0.0455      0.048      0.939      0.348      -0.050       0.141
Access_to_exercise_opportunities_raw_value                                         -0.0455      0.035     -1.286      0.199      -0.115       0.024
Excessive_drinking_raw_value                                                       -0.0819      0.047     -1.740      0.082      -0.174       0.010
Alcohol_impaired_driving_deaths_raw_value                                           0.0245      0.026      0.937      0.349      -0.027       0.076
Sexually_transmitted_infections_raw_value                                          -0.0010      0.037     -0.026      0.979      -0.073       0.071
Teen_births_raw_value                                                               0.0961      0.053      1.797      0.073      -0.009       0.201
Uninsured_raw_value                                                                 3.1106      0.645      4.822      0.000